In [2]:
import keras

model_valence = keras.models.load_model("D:/val_model/valence_model")
model_arousal = keras.models.load_model("D:/aro_model/arousal_model")
model_dominance = keras.models.load_model("D:/dom_model/dominance_model")

In [3]:
import pandas as pd

df = pd.read_csv("D:\\Marko\\Project_NLP_171161\\text_emotion.csv", 
           names=['tweet_id', 'target', 'author', 'text'],
           header=0)
df.head()

,tweet_id,target,author,text
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [4]:
xy_df = pd.DataFrame(df.values[:,1:4:2], columns=['labels', 'text'])

xy_df.head()

,labels,text
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
from ipynb.fs.full.preprocessing_text import PreprocessTweet
from ipynb.fs.full.dict_lexicons import read_lexicon
from ipynb.fs.full.lexicon_mapping import LexiconMapper, Analyzer
from ipynb.fs.full.bert_helper import create_raw_input, pull_bert, get_max_lengths, bert_encode

preprocessor = PreprocessTweet()
preprocessed_df = preprocessor.preprocess_tweets(xy_df, False, True, False, True)
preprocessed_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Label,Tweet,Mark,Caps,Letters
0,empty,i know i was listenin to bad habit earlier and...,False,False,False
1,sadness,layin n bed with a headache ughhhh waitin on y...,False,False,True
2,sadness,funeral ceremony gloomy friday,False,False,True
3,enthusiasm,wants to hang out with friends soon,True,False,False
4,neutral,we want to trade with someone who has houston ...,False,False,False


In [6]:
def init_transfered_model(df, attr, size):
    bert_layer, tokenizer = pull_bert('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/2')
    X_train = bert_encode(df['Tweet'], tokenizer, size)
    
    return X_train, bert_layer

X_train, bert_layer = init_transfered_model(preprocessed_df, 'Label', 70)
print(X_train)

{'input_word_ids': <tf.Tensor: shape=(40000, 70), dtype=int32, numpy=
array([[  101,  1045,  2113, ...,     0,     0,     0],
       [  101,  3913,  2378, ...,     0,     0,     0],
       [  101,  6715,  5103, ...,     0,     0,     0],
       ...,
       [  101,  3407, 10756, ...,     0,     0,     0],
       [  101,  2001,  6342, ...,     0,     0,     0],
       [  101,  7960,  3345, ...,     0,     0,     0]])>, 'input_mask': <tf.Tensor: shape=(40000, 70), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'input_type_ids': <tf.Tensor: shape=(40000, 70), dtype=int32, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])>}


In [7]:
import numpy as np

predicted_valence = model_valence.predict(X_train)
preprocessed_df['valence'] = [str(i) for i in predicted_valence]

In [9]:
from collections import Counter

preprocessed_df['valence'] = [str(i) for i in predicted_valence]
labels_val = []
for i in preprocessed_df.values:
    avg_arr = [float(j) for j in i[5][1:-1].split()]
    if True:
        m = np.argmax(avg_arr)
        if m == 1:
            labels_val.append("high")
        elif m == 0:
            labels_val.append("low")
    else:
        labels_val.append("Na")
print(Counter(labels_val)) 
preprocessed_df['valence_label'] = labels_val
preprocessed_df[preprocessed_df.valence_label == "high"]

Counter({'low': 23018, 'high': 16982})


,Label,Tweet,Mark,Caps,Letters,valence,valence_label
14,surprise,got the news,False,False,False,[0.39432675 0.6056733 ],high
16,love,agreed,False,False,False,[0.14100449 0.8589955 ],high
18,worry,lady gaga tweeted about not being impressed by...,False,False,False,[0.37489843 0.62510157],high
40,happiness,mmm much better day so far its still quite ear...,True,False,True,[0.43286836 0.56713164],high
41,fun,cause your amazing lol come to canada would do...,False,False,False,[0.14211038 0.85788965],high
...,...,...,...,...,...,...,...
39994,happiness,succesfully following tayla,True,False,False,[0.36875576 0.6312442 ],high
39995,neutral,,False,False,False,[0.13000667 0.8699933 ],high
39996,love,happy mothers day all my love,False,False,False,[0.02406326 0.9759368 ],high
39997,love,happy mothers day to all the mommies out there...,True,False,False,[0.13616166 0.8638383 ],high


In [10]:
predicted_arousal = model_arousal.predict(X_train)
preprocessed_df['arousal'] = [str(i) for i in predicted_arousal]

In [11]:
from collections import Counter
import numpy as np

preprocessed_df['arousal'] = [str(i) for i in predicted_arousal]
labels_aro = []
for i in preprocessed_df.values:
    avg_arr = [float(j) for j in i[7][1:-1].split()]
    if True:
        m = np.argmax(avg_arr)
        if m == 0:
            labels_aro.append("high")
        elif m == 1:
            labels_aro.append("low")
    else:
        labels_aro.append("Na")
print(Counter(labels_aro)) 
preprocessed_df['arousal_label'] = labels_aro
preprocessed_df[preprocessed_df.arousal_label == "high"]

Counter({'low': 21281, 'high': 18719})


,Label,Tweet,Mark,Caps,Letters,valence,valence_label,arousal,arousal_label
0,empty,i know i was listenin to bad habit earlier and...,False,False,False,[0.93751884 0.06248121],low,[0.9972397 0.00276026],high
1,sadness,layin n bed with a headache ughhhh waitin on y...,False,False,True,[0.9552764 0.04472362],low,[0.5553313 0.44466874],high
4,neutral,we want to trade with someone who has houston ...,False,False,False,[0.9165335 0.08346643],low,[0.9890258 0.01097426],high
6,sadness,i should be sleep but im not thinking about an...,True,True,False,[0.93562573 0.06437422],low,[0.9926519 0.0073482],high
8,sadness,charlene my love i miss you,False,True,False,[0.7734836 0.22651646],low,[0.99817955 0.0018205 ],high
...,...,...,...,...,...,...,...,...,...
39988,happiness,had such and amazing time last night mcfly wer...,False,True,False,[0.03166995 0.96833 ],high,[9.9932396e-01 6.7600969e-04],high
39989,love,his snoring is so annoying n it keeps me from ...,False,True,False,[0.9322233 0.06777669],low,[0.9963709 0.0036291],high
39996,love,happy mothers day all my love,False,False,False,[0.02406326 0.9759368 ],high,[0.99803823 0.0019618 ],high
39998,happiness,wassup beautiful follow me peep out my new hit...,True,True,True,[0.04371156 0.95628846],high,[0.9953961 0.00460387],high


In [12]:
predicted_dominance = model_dominance.predict(X_train)
preprocessed_df['dominance'] = [str(i) for i in predicted_dominance]

In [13]:
from collections import Counter

preprocessed_df['dominance'] = [str(i) for i in predicted_dominance]
labels_dom = []
for i in preprocessed_df.values:
    avg_arr = [float(j) for j in i[9][1:-1].split()]
    if True:
        m = np.argmax(avg_arr)
        if m == 0:
            labels_dom.append("high")
        elif m == 1:
            labels_dom.append("low")
    else:
        labels_dom.append("Na")
print(Counter(labels_dom)) 
preprocessed_df['dominance_label'] = labels_dom
preprocessed_df[preprocessed_df.dominance_label == "high"]

Counter({'high': 28213, 'low': 11787})


,Label,Tweet,Mark,Caps,Letters,valence,valence_label,arousal,arousal_label,dominance,dominance_label
0,empty,i know i was listenin to bad habit earlier and...,False,False,False,[0.93751884 0.06248121],low,[0.9972397 0.00276026],high,[9.9955887e-01 4.4114422e-04],high
3,enthusiasm,wants to hang out with friends soon,True,False,False,[0.647147 0.35285303],low,[0.39248502 0.607515 ],low,[9.9954295e-01 4.5703768e-04],high
4,neutral,we want to trade with someone who has houston ...,False,False,False,[0.9165335 0.08346643],low,[0.9890258 0.01097426],high,[9.996244e-01 3.756004e-04],high
5,worry,re pinging why didnt you go to prom bc my bf d...,False,True,False,[0.94179004 0.05820993],low,[0.02396669 0.97603333],low,[0.9751923 0.02480765],high
6,sadness,i should be sleep but im not thinking about an...,True,True,False,[0.93562573 0.06437422],low,[0.9926519 0.0073482],high,[9.9981135e-01 1.8864145e-04],high
...,...,...,...,...,...,...,...,...,...,...,...
39994,happiness,succesfully following tayla,True,False,False,[0.36875576 0.6312442 ],high,[0.0025858 0.99741423],low,[0.85302025 0.14697972],high
39996,love,happy mothers day all my love,False,False,False,[0.02406326 0.9759368 ],high,[0.99803823 0.0019618 ],high,[9.9957770e-01 4.2224253e-04],high
39997,love,happy mothers day to all the mommies out there...,True,False,False,[0.13616166 0.8638383 ],high,[0.20843841 0.7915616 ],low,[9.9955589e-01 4.4404963e-04],high
39998,happiness,wassup beautiful follow me peep out my new hit...,True,True,True,[0.04371156 0.95628846],high,[0.9953961 0.00460387],high,[0.9984602 0.00153988],high


In [14]:
labels = []
for i in preprocessed_df.values:
    if i[6] == 'low' and i[8] == 'low' and i[10] == 'low':
        labels.append('sadness')
    elif i[6] == 'high' and i[8] == 'low' and i[10] == 'low':
        labels.append('fear')
    elif i[6] == 'high' and i[8] == 'high' and i[10] == 'low':
        labels.append('anger')
    elif i[6] == 'high' and i[8] == 'high' and i[10] == 'high':
        labels.append('joy')
    elif i[6] == 'high' and i[8] == 'low' and i[10] == 'high':
        labels.append('surprise')
    elif i[6] == 'low' and i[8] == 'low' and i[10] == 'high':
        labels.append('happiness')
    elif i[6] == 'low' and i[8] == 'high' and i[10] == 'low':
        labels.append('disgust')
    elif i[6] == 'low' and i[8] == 'high' and i[10] == 'high':
        labels.append('happiness')
    else:
        labels.append("NONE")
#print(labels)
#print(len(labels))
preprocessed_df['label'] = labels
preprocessed_df.head(20)
Counter(preprocessed_df.label)

Counter({'happiness': 15804,
         'disgust': 2551,
         'sadness': 4663,
         'surprise': 5796,
         'fear': 4067,
         'joy': 6613,
         'anger': 506})

In [15]:
preprocessed_df[preprocessed_df.label == 'joy']

,Label,Tweet,Mark,Caps,Letters,valence,valence_label,arousal,arousal_label,dominance,dominance_label,label
18,worry,lady gaga tweeted about not being impressed by...,False,False,False,[0.37489843 0.62510157],high,[0.93874264 0.06125734],high,[9.9937963e-01 6.2035990e-04],high,joy
41,fun,cause your amazing lol come to canada would do...,False,False,False,[0.14211038 0.85788965],high,[9.995881e-01 4.118889e-04],high,[9.9980134e-01 1.9862803e-04],high,joy
66,worry,ia so much i havent really been happy with any...,True,False,False,[0.46837354 0.5316265 ],high,[9.995851e-01 4.149240e-04],high,[9.9906117e-01 9.3882193e-04],high,joy
71,worry,and all woman who transfer their first impress...,False,False,True,[0.40206593 0.59793407],high,[0.9905165 0.00948345],high,[0.9942433 0.00575664],high,joy
72,neutral,brothers bloom wont be opening this weekend in...,False,True,False,[0.16258502 0.837415 ],high,[0.99688184 0.00311818],high,[0.99802566 0.00197433],high,joy
...,...,...,...,...,...,...,...,...,...,...,...,...
39979,surprise,awesome whadya get her,False,False,False,[0.02657708 0.9734229 ],high,[0.9988527 0.00114728],high,[9.9976915e-01 2.3089298e-04],high,joy
39985,happiness,husband is golfing the toddler and i shall fro...,False,True,False,[0.07282974 0.9271702 ],high,[0.99596536 0.00403468],high,[9.991436e-01 8.563746e-04],high,joy
39988,happiness,had such and amazing time last night mcfly wer...,False,True,False,[0.03166995 0.96833 ],high,[9.9932396e-01 6.7600969e-04],high,[9.9983907e-01 1.6093113e-04],high,joy
39996,love,happy mothers day all my love,False,False,False,[0.02406326 0.9759368 ],high,[0.99803823 0.0019618 ],high,[9.9957770e-01 4.2224253e-04],high,joy


In [19]:
import csv
file = open('newlabels.csv', 'w+', encoding="utf-8")
dataWriter = csv.writer(file, delimiter=',',lineterminator="\n")

total = 40000
[dataWriter.writerow([preprocessed_df.values[i,1], preprocessed_df.values[i,11]]) for i in range(total)]
print('Completed preprocessing!')
file.close()

Completed preprocessing!
